<a href="https://colab.research.google.com/github/KlausAMF/glusiosai/blob/main/Glusios_Assistente_Financeiro_Pessoal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1. Instalando o SDK do Google

In [36]:
!pip install -q -U google-generativeai

Configurações iniciais

In [37]:
import google.generativeai as genai
from pathlib import Path
import hashlib
import csv
from google.colab import userdata

API_KEY = userdata.get('api_key')

GOOGLE_API_KEY = API_KEY
genai.configure(api_key=GOOGLE_API_KEY)

## 2. Preparação do modelo

In [38]:
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 0,
  "max_output_tokens": 8192,
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_LOW_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_LOW_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_LOW_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_LOW_AND_ABOVE"
  },
]

In [39]:
system_instruction = "Você é um assistente de finanças pessoais, mais especificamente voltado para análise de despesas pessoais. Você conversa com pessoas leigas e que precisam da sua ajuda para otimizar seus gastos e saber como categorizá-los. Você faz cálculos precisos.\n\nSeu papel é tornar o planejamento financeiro pessoal fácil e divertido para pessoas de forma segura, prática e totalmente livre de \"tabus financeiros\".\n\nO usuário que te perguntar algo precisa entender bem o que você responde.\n\nSeu nome é Glusios. Refira-se a si próprio como \"o seu assistente de gastos pessoais\"."

In [40]:
model = genai.GenerativeModel(model_name="gemini-1.5-pro-latest",
                              generation_config=generation_config,
                              system_instruction=system_instruction,
                              safety_settings=safety_settings)

## 3. Desenvolvimento do Glusios, seu assistente financeiro pessoal

In [41]:
def extract_pdf_pages(pathname: str) -> list[str]:
  parts = [f"--- START OF PDF ${pathname} ---"]
  # Add logic to read the PDF and return a list of pages here.
  pages = []
  for index, page in enumerate(pages):
    parts.append(f"--- PAGE {index} ---")
    parts.append(page)
  return parts

extract_pdf_pages("/content/despesas .csv")

['--- START OF PDF $/content/despesas .csv ---']

In [42]:
def extract_csv_data(pathname: str) -> list[str]:

  parts = [f"--- START OF CSV {pathname} ---"]
  data = []

  # Open the CSV file in read mode
  with open(pathname, 'r') as csvfile:
    # Create a CSV reader object
    reader = csv.reader(csvfile)

    # Iterate through each row in the CSV file
    for row in reader:
      # Append each row as a string to the data list
      data.append(','.join(row))

  parts.extend(data)
  parts.append("--- END OF CSV ---")
  return parts


In [43]:
convo = model.start_chat(history=[
  {
    "role": "user",
    "parts": extract_csv_data("/content/despesas .csv")
  }
])

## 4. Chat

In [ ]:
print("Glusios: Olá! Sou o Glusios, o seu assistente de gastos pessoais. Que legal ter você por aqui! Vejo que você trouxe um registro bem detalhado das suas despesas. Isso é um ótimo começo para entendermos para onde seu dinheiro está indo. 😄 \nPara começarmos nossa análise, me fale: o que te intriga ou te preocupa em relação a esses gastos? 🤔\nPor exemplo:\nVocê gostaria de identificar as áreas onde gasta mais?\nQuer saber se está gastando muito com presentes?\nDeseja ter uma visão geral de como suas despesas se dividem entre as categorias?\nCom essas informações, posso te ajudar de forma mais precisa.\nPara encerrar a conversa, digite fim\n")

# Chat
prompt = input('Fale com o Glusios: ')

while prompt != "fim":
  response = convo.send_message(prompt)
  print("---\n\nGlusios: ", response.text, '\n\n---\n\n')
  prompt = input('Fale com o Glusios: ')